In [3]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB 991.0 kB/s eta 0:00:15
   - -------------------------------------- 0.6/14.3 MB 7.7 MB/s eta 0:00:02
   ---- ----------------------------------- 1.5/14.3 MB 11.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.7/14.3 MB 15.4 MB/s eta 0:00:01
   ----------- ---------------------------- 4.1/14.3 MB 18.9 MB/s eta 0:00:01
   ---------------- ----------------------- 6.1/14.3 MB 22.9 MB/s eta 0:00:01
   --------------------- ------------------ 7.6/14.3 MB 24.3 MB/s eta 0:00:01
   ---------------------- ----------------- 8.0/14.3 MB 22.3 MB/s eta 0:00:01
   ---------------------------- ----------- 10.3/14.3 MB 28.5 MB/s eta 0:00:01
   ----------------------------------- ---- 12.8/14.3 MB 36.4 MB/s eta 0:00:01
   ------------------------------------ --- 12.9/14.3 MB 32.7 MB/s eta 0:00:01
   ------------------------------------- -- 13.3/14.3 MB 28.4 MB/s et

In [13]:
# TMDB API키로 영화정보 장르 데이터베이스에 저장하기
# movies 테이블과 movie_genres 테이블에 데이터 추가

import requests
import mysql.connector

# TMDb API 키 설정
API_KEY = '152131ad224af4e20571dd70deab9550'  # TMDb API 키를 여기에 입력하세요

# 데이터베이스 연결 설정
db_config = {
    'user': 'mymy',
    'password': '321',
    'host': 'project-db-cgi.smhrd.com',
    'port': 3307,
    'database': 'mymy'  # 실제 데이터베이스 이름으로 변경
}

# TMDb API로 영화 데이터 가져오기
def fetch_movies(page=1):
    url = f'https://api.themoviedb.org/3/movie/popular?api_key={API_KEY}&language=en-US&page={page}'
    response = requests.get(url)
    data = response.json()
    if 'results' in data:
        return data
    else:
        print("No 'results' key found in response")
        return {'results': [], 'total_pages': 1}

# 데이터베이스에 영화 삽입
def insert_movies(movies):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    for movie in movies:
        cursor.execute(
            """
            INSERT IGNORE INTO movies (movie_idx, movie_title_en, movie_title_kr, movie_playtime, opened_at, movie_poster_url, overview)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """,
            (movie['id'], movie.get('title', ''), movie.get('original_title', ''), 
             movie.get('runtime', ''), movie.get('release_date', ''), 
             f"https://image.tmdb.org/t/p/w500{movie.get('poster_path', '')}",
             movie.get('overview', ''))
        )
    conn.commit()
    cursor.close()
    conn.close()

# 데이터베이스에 영화 장르 관계 삽입
def insert_movie_genres(movies):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    for movie in movies:
        movie_id = movie['id']
        for genre_id in movie.get('genre_ids', []):
            cursor.execute("INSERT IGNORE INTO movie_genres (movie_idx, genre_idx) VALUES (%s, %s)", (movie_id, genre_id))
    conn.commit()
    cursor.close()
    conn.close()

# 메인 함수
def main():
    # 전체 페이지를 반복하여 영화 데이터 가져오기 및 삽입
    page = 1
    while True:
        data = fetch_movies(page)
        movies = data['results']
        total_pages = data['total_pages']
        
        # 영화 데이터를 삽입
        insert_movies(movies)
        insert_movie_genres(movies)

        # 모든 페이지를 가져왔는지 확인
        if page >= total_pages:
            break
        page += 1

if __name__ == "__main__":
    main()


No 'results' key found in response


In [14]:
# 영화 한글제목 추가

import requests
import mysql.connector

# TMDb API 키 설정
API_KEY = '152131ad224af4e20571dd70deab9550'

# 데이터베이스 연결 설정
db_config = {
    'user': 'mymy',
    'password': '321',
    'host': 'project-db-cgi.smhrd.com',
    'port': 3307,
    'database': 'mymy'
}

# TMDb API로 영화 데이터 가져오기 (한글 제목을 가져오기 위해 language=ko-KR 설정)
def fetch_movies(page=1):
    url = f'https://api.themoviedb.org/3/movie/popular?api_key={API_KEY}&language=ko-KR&page={page}'
    response = requests.get(url)
    data = response.json()
    if 'results' in data:
        return data['results']
    else:
        print("No 'results' key found in response")
        return []

# 데이터베이스에 영화 한글 제목 업데이트
def update_movie_korean_titles(movies):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    for movie in movies:
        # 한글 제목이 없는 경우 영어 제목을 사용합니다.
        korean_title = movie.get('title', movie.get('original_title', ''))
        cursor.execute(
            """
            UPDATE movies
            SET movie_title_kr = %s
            WHERE movie_idx = %s
            """,
            (korean_title, movie['id'])
        )
    conn.commit()
    cursor.close()
    conn.close()

# 메인 함수
def main():
    # 전체 페이지를 반복하여 영화 데이터 가져오기 및 업데이트
    page = 1
    while True:
        movies = fetch_movies(page)
        if not movies:
            break
        update_movie_korean_titles(movies)
        page += 1

if __name__ == "__main__":
    main()


No 'results' key found in response


In [15]:
# 배우 테이블 데이터 저장

import requests
import mysql.connector

# TMDb API 키 설정 (본인의 API 키로 변경)
API_KEY = '152131ad224af4e20571dd70deab9550'

# 데이터베이스 연결 설정
db_config = {
    'user': 'mymy',
    'password': '321',
    'host': 'project-db-cgi.smhrd.com',
    'port': 3307,
    'database': 'mymy'  # 실제 데이터베이스 이름으로 변경
}

# TMDb API로 배우 데이터 가져오기
def fetch_actors(page=1):
    url = f'https://api.themoviedb.org/3/person/popular?api_key={API_KEY}&language=ko-KR&page={page}'
    response = requests.get(url)
    data = response.json()
    if 'results' in data:
        return data['results']
    else:
        print("No 'results' key found in response")
        return []

# 데이터베이스에 배우 삽입
def insert_actors(actors):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    for actor in actors:
        cursor.execute(
            """
            INSERT IGNORE INTO actors (actor_idx, actor_name, actor_profile_url)
            VALUES (%s, %s, %s)
            """,
            (actor.get('id', ''),  # 배우의 TMDb ID (actor_idx로 저장)
             actor.get('name', ''),  # 배우의 이름
             f"https://image.tmdb.org/t/p/w500{actor.get('profile_path', '')}" if actor.get('profile_path') else '')  # 배우의 프로필 사진 URL
        )
    conn.commit()
    cursor.close()
    conn.close()

# 메인 함수
def main():
    page = 1
    while True:
        actors = fetch_actors(page)
        if not actors:
            break
        insert_actors(actors)
        page += 1

if __name__ == "__main__":
    main()


No 'results' key found in response


In [4]:
# 캐스팅 정보 저장

import requests
import mysql.connector

# TMDb API 키 설정 (본인의 API 키로 변경)
API_KEY = '152131ad224af4e20571dd70deab9550'

# 데이터베이스 연결 설정
db_config = {
    'user': 'mymy',
    'password': '321',
    'host': 'project-db-cgi.smhrd.com',
    'port': 3307,
    'database': 'mymy'  # 실제 데이터베이스 이름으로 변경
}

# 범위 내 영화 ID 설정
START_MOVIE_ID = 176779
END_MOVIE_ID = 200000

# TMDb API로 영화 캐스팅 정보 가져오기
def fetch_castings(movie_id):
    url = f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}&language=ko-KR'
    response = requests.get(url)
    data = response.json()
    if 'cast' in data:
        return data['cast']
    else:
        print(f"No 'cast' key found for movie ID: {movie_id}")
        return []

# 데이터베이스에 캐스팅 정보 삽입
def insert_castings(movie_id, castings):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    for index, casting in enumerate(castings):
        actor_id = casting.get('id', '')
        character_name = casting.get('character', '')
        cursor.execute(
            """
            INSERT IGNORE INTO castings (movie_idx, actor_idx, character_name, casting_order)
            VALUES (%s, %s, %s, %s)
            """,
            (movie_id, actor_id, character_name, index + 1)  # index는 0부터 시작하므로 +1을 해서 출연 순서를 나타냄
        )
    conn.commit()
    cursor.close()
    conn.close()

# 메인 함수
def main():
    for movie_id in range(START_MOVIE_ID, END_MOVIE_ID + 1):
        castings = fetch_castings(movie_id)
        if castings:
            insert_castings(movie_id, castings)
            print(f"Castings for Movie ID {movie_id} successfully inserted.")
        else:
            print(f"No castings found for Movie ID {movie_id}.")

if __name__ == "__main__":
    main()


No castings found for Movie ID 176779.
No castings found for Movie ID 176780.
No 'cast' key found for movie ID: 176781
No castings found for Movie ID 176781.
No 'cast' key found for movie ID: 176782
No castings found for Movie ID 176782.
Castings for Movie ID 176783 successfully inserted.
No 'cast' key found for movie ID: 176784
No castings found for Movie ID 176784.
No 'cast' key found for movie ID: 176785
No castings found for Movie ID 176785.
Castings for Movie ID 176786 successfully inserted.
No 'cast' key found for movie ID: 176787
No castings found for Movie ID 176787.
Castings for Movie ID 176788 successfully inserted.
No 'cast' key found for movie ID: 176789
No castings found for Movie ID 176789.
Castings for Movie ID 176790 successfully inserted.
Castings for Movie ID 176791 successfully inserted.
No 'cast' key found for movie ID: 176792
No castings found for Movie ID 176792.
Castings for Movie ID 176793 successfully inserted.
Castings for Movie ID 176794 successfully inserted